# Clase Supervised Classification ML

## Importar librerías

In [36]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OrdinalEncoder, OneHotEncoder
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
from joblib import dump, load
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier



## Importar los datos

In [ ]:
# Tu código aquí debajo
df = pd.read_excel('diabetes_clean.xlsx')
df.sample(8)

## Detectar y tratar duplicados

In [3]:
# Tu código aquí debajo
df.duplicated().sum()


0

## Detectar y tratar valores nulos

In [4]:
# Tu código aquí debajo
df.isnull().sum()


Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Satisfaction                0
Job                         0
Outcome                     0
dtype: int64

In [ ]:
# Tu código aquí debajo



## Detectar y tratar valores atípicos (ouliers)

In [5]:
# Tu código aquí debajo
numericas=['Pregnancies','Glucose','BloodPressure','SkinThickness','Insulin','BMI','DiabetesPedigreeFunction','Age']

# FUNCIÓN QUE DEVUELVE LOS NÚMEROS DE FILA EN LOS QUE LA VARIABLE EN CUESTIÓN ES OULIER
def outliers(var):
  q1=var.quantile(0.25)
  q3=var.quantile(0.75)
  riq=q3-q1
  sup=q3+1.5*(riq)
  inf=q1-1.5*(riq)
  outl=(var>sup) | (var<inf)
  return outl

for i in df.loc[:,numericas]:
  print("Cantida de oultliers de la variable",i,":",outliers(df[i]).sum())
# Eliminar outliers
# Tu código
for i in df.loc[:,numericas]:
  df=df[~outliers(df[i])]
df.sample(5)

Cantida de oultliers de la variable Pregnancies : 0
Cantida de oultliers de la variable Glucose : 9
Cantida de oultliers de la variable BloodPressure : 4
Cantida de oultliers de la variable SkinThickness : 0
Cantida de oultliers de la variable Insulin : 4
Cantida de oultliers de la variable BMI : 4
Cantida de oultliers de la variable DiabetesPedigreeFunction : 10
Cantida de oultliers de la variable Age : 10


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Satisfaction,Job,Outcome
586,8,120,86,0,0,28.4,259.0,22,Unsatisfied,Legislator,1
98,2,107,74,30,100,33.6,404.0,23,Extremely satisfied,High School Teacher,0
559,4,127,88,11,155,34.5,598.0,28,Extremely satisfied,Human Resources,0
444,1,84,64,23,115,36.9,471.0,28,Extremely satisfied,Legislator,0
511,13,104,72,0,0,31.2,465.0,38,Unsatisfied,High School Teacher,1


# Preparamos los datos



## Transformar variables ordinales en numéricas

Como vemos tenemos la variable Satisfaction que es una variable ordinal y podemos suponer que el impacto que podrá tener sobre la variable a predecir (diabetes si/no) permanecerá constante al pasar de una categoría a la siguiente. por eso, la codificaremos como variable ordinal

In [6]:
# Creamos el objeto que realizará la transformación dándole el orden en una lista con doble corchete
enc=OrdinalEncoder(categories=[['Extremely unsatisfied','Unsatisfied','Satisfied','Extremely satisfied']],dtype='int')
# Particularizamos el codificador en nuestros datos
# Tu código
enc.fit(df[['Satisfaction']])
# Aplicamos el codificador ya particularizado a nuestros datos y lo guardamos machacando la misma variable
# Tu código
df['Satisfaction']=enc.transform(df[['Satisfaction']])
print(df.shape)
df.sample(5)


(576, 11)


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Satisfaction,Job,Outcome
137,3,111,90,12,78,28.4,495.0,29,1,Market Research Analyst,0
204,0,86,68,32,0,35.8,238.0,25,3,High School Teacher,0
574,5,97,76,27,0,35.6,378.0,52,1,Human Resources,1
153,3,111,62,0,0,22.6,142.0,21,3,Human Resources,0
496,2,68,62,13,15,20.1,257.0,23,1,High School Teacher,0


## Transformar variables nominales en numéricas

Como vemos tenemos la variable Job que es una variable nominal (sin orden) y debenmos codificarla mediante el One-Hot encoder

In [7]:
# Tu código
# Creamos el objeto que realizará la transformación

#Instancio el OneHot
onehot=OneHotEncoder()

# Aplico el OneHot a la columna job y guardo el resultao en a
a=onehot.fit_transform(df[['Job']])

# Convierto el a en data frame y lo llamo encoded_df
encoded_df = pd.DataFrame(a.toarray(), columns=onehot.get_feature_names_out(['Job']))

# Reseteo el índice de los dos data frames antes de concatenarlos
df.reset_index(drop=True, inplace=True)
encoded_df.reset_index(drop=True, inplace=True)

# concateno los dos data frames y los guardo machacando el df
df=pd.concat([df,encoded_df],axis=1)

# Elimino la columna Job original antigua, la categórica
df.drop(columns='Job',inplace=True)

# Muestro tamaño yu sample del data frame df transformado
print(df.shape)
df.sample(5)



(576, 20)


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Satisfaction,Outcome,Job_Database Administrator,Job_Epidemiologist,Job_Financial Analyst,Job_Firefighter,Job_High School Teacher,Job_Human Resources,Job_Legislator,Job_Market Research Analyst,Job_Physicians,Job_Statistician
515,3,130,64,0,0,23.1,314.00,22,3,0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
401,0,107,76,0,0,45.3,686.00,24,3,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
490,11,136,84,35,130,28.3,0.26,42,2,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
452,1,108,88,19,0,27.1,0.40,24,2,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
553,0,126,86,27,120,27.4,515.00,21,0,0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


## 0. Separar la variable a predecir y las predictoras

In [8]:
X=df.drop(columns=['Outcome'],inplace=False)
y=df['Outcome']


## 1. Split. Separar los datos en conjunto de entrenamiento (train) y conjunto de evaluación o test(test)

In [9]:
# Tu código aquí debajo
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)

## 2. Estandarización controlada.

* Fit en el train
* Transform en el train
* Transform en el test

In [10]:
# Imprime la dimensión de los datos que vayas creando
# Tu código aquí debajo
estandarizador = StandardScaler()
estandarizador.fit(X_train)
X_train_std=estandarizador.transform(X_train)
X_test_std=estandarizador.transform(X_test)

# REGRESIÓN LOGÍSTICA. Usamos la estandarización



In [16]:
LR = LogisticRegression()
LR.fit(X_train_std,y_train)
predictions = LR.predict(X_test_std)

In [20]:
metricas_LR=[]
metricas_LR.append(accuracy_score(y_test, predictions))
metricas_LR.append(precision_score(y_test, predictions))
metricas_LR.append(recall_score(y_test, predictions))
metricas_LR.append(f1_score(y_test, predictions))
metricas_LR

[0.8055555555555556,
 0.5925925925925926,
 0.48484848484848486,
 0.5333333333333333]

# DECISION TREE. NO Usamos la estandarización

In [22]:
DT=DecisionTreeClassifier()
DT.fit(X_train,y_train)
predictions = DT.predict(X_test)
metricas_DT=[]
metricas_DT.append(accuracy_score(y_test, predictions))
metricas_DT.append(precision_score(y_test, predictions))
metricas_DT.append(recall_score(y_test, predictions))
metricas_DT.append(f1_score(y_test, predictions))
metricas_DT



[0.7847222222222222, 0.53125, 0.5151515151515151, 0.5230769230769231]

# RANDOM FOREST. Usamos la estandarización

In [40]:
RF=RandomForestClassifier()
RF.fit(X_train_std,y_train)
predictions = RF.predict(X_test_std)
metricas_RF=[]
metricas_RF.append(accuracy_score(y_test, predictions))
metricas_RF.append(precision_score(y_test, predictions))
metricas_RF.append(recall_score(y_test, predictions))
metricas_RF.append(f1_score(y_test, predictions))
metricas_RF

[0.8055555555555556, 0.6086956521739131, 0.42424242424242425, 0.5]

In [41]:
dict={"Logistic Regression":metricas_LR, "DecisionTree":metricas_DT, "RandomForest":metricas_RF}
Resultados=pd.DataFrame(dict,index=["Acuracy","Precision","Recall","F1"])
Resultados

,Logistic Regression,DecisionTree,RandomForest
Acuracy,0.805556,0.784722,0.805556
Precision,0.592593,0.531250,0.608696
Recall,0.484848,0.515152,0.424242
F1,0.533333,0.523077,0.500000
